In [144]:
import nltk
import string
from nltk import pos_tag
import spacy
import numpy as np
from nltk import grammar, parse
from collections import Counter 
from cleantext import clean
import re

In [145]:
nlp = spacy.load("en")

In [146]:
filename = '../data/test2.txt'
f = open(filename, "r")
contents = f.read()

clean("some input",
    fix_unicode=True,               # fix various unicode errors
    to_ascii=True,                  # transliterate to closest ASCII representation
    lower=True,                     # lowercase text
    no_line_breaks=False,           # fully strip line breaks as opposed to only normalizing them
    no_urls=False,                  # replace all URLs with a special token
    no_emails=False,                # replace all email addresses with a special token
    no_phone_numbers=False,         # replace all phone numbers with a special token
    no_numbers=False,               # replace all numbers with a special token
    no_digits=False,                # replace all digits with a special token
    no_currency_symbols=False,      # replace all currency symbols with a special token
    no_punct=False,                 # fully remove punctuation
    replace_with_url="",
    replace_with_email="",
    replace_with_phone_number="",
    replace_with_number="",
    replace_with_digit="",
    replace_with_currency_symbol="",
    lang="en"                       # set to 'de' for German special handling
)

contents = re.sub('i.e.', 'ie', contents)
contents = re.sub(r'\d+', '', contents)
contents = re.sub(r"""
               [,;@#?!&$\"\'\[\]]+  # Accept one or more copies of punctuation
               \ *           # plus zero or more copies of a space,
               """,
               " ",          # and replace it with a single space
               contents, flags=re.VERBOSE)
contents = re.sub(' +', ' ', contents)

In [147]:
utterances = []
split_utterances = []
sentences = contents.strip().split('.')
for sentence in sentences:
    if sentence != '':
        utterances.append(sentence.strip())

In [148]:
## pos tags
pos_tags_utterances = []

for utterance in utterances:
    pos_tags_utterances.append(pos_tag(utterance.split(' ')))

By Marvel Entertainment began planning to independently produce its own films and distribute them through Paramount Pictures
Previously Marvel had co-produced several superhero films with Columbia Pictures New LieCiea and others including a seven-year development deal with th Century Fox
Marvel made relatiey little profit from its liesing deals with other studios and wanted to get more money out of its films whiemaintaining artistic control of the projects and distribution
Avi Arad head of Marvel s film division was pleased with Sam Raimi s Spie-Man films at Sony but was less pleased about others
As a result they decie to form Marvel Studios Hollywood s first major independent movie studio since DreamWorks
Arad s second-in-command Kevin Feie realie that unlieSpie-Man and the X-Men whose film rights were liesed to Sony and Fox respectiey Marvel still owned the rights to the core members of the Avengers
Feie a self-professed fanboy envisioned creating a shared uniese just as creators Sta

In [149]:
# topic of first sentence
processed_utterance = nlp(utterances[0])
init_topic = ''
for chunk in processed_utterance.noun_chunks:
    if chunk.root.dep_ == "nsubj":
        init_topic = chunk.text

In [150]:
# Centers of 
total_utterances = len(utterances)

Cb = [None] * total_utterances

Cf = []
for i in range(total_utterances):
    Cf.append([])

Cb[0] = "undefined"
for i in range(total_utterances):
    
    for token in pos_tags_utterances[i]:
        if "NN" in token[1]:
            Cf[i].append((token[0].lower(), token[1]))
        if i!=0:
            if "PRP" == token[1]:
                if token[0].lower() == "he" or token[0].lower() == "she":
                    for pos in Cf[i-1]:
                        if pos[1] == "NNP":
                            if (pos[0].lower(), "NNP") not in Cf[i]:
                                Cf[i].append((pos[0].lower(), "NNP"))
                                break
                elif token[0].lower() == "it":
                    for pos in Cf[i-1]:
                        if pos[1] == "NN":
                            if (pos[0].lower(), "NN") not in Cf[i]:
                                Cf[i].append((pos[0].lower(), "NN"))
                                break
                elif token[0].lower() == 'they':
                    for pos in Cf[i-1]:
                        if pos[1] == "NNS":
                            if (pos[0].lower(), "NN") not in Cf[i]:
                                Cf[i].append((pos[0].lower(), "NNS"))
                                break

    if i != 0:
        try:
            Cb[i] = Cf[i][0]
        except:
            Cb[i] = Cb[i-1]

In [151]:
topics_utterances = []
focus_utterances = []
topics_utterances.append(init_topic)
for tuple_val in Cb:
    if tuple_val != "undefined":
        if tuple_val[1] == "NNP":
            topics_utterances.append(tuple_val[0].capitalize())
        else:
            topics_utterances.append(tuple_val[0])

In [152]:
topic_dict = {}
for topic in topics_utterances:
    if topic not in topic_dict.keys():
        topic_dict[topic] = 1
    else:
        topic_dict[topic] += 1
        
k  = Counter(topic_dict)
top_topics_num = len(topic_dict)//2
top_topics_num = 4
top_topics_list = k.most_common(top_topics_num)


print("Topics present in the discourse are:")
for topic in top_topics_list:
    print(topic[0])

Topics present in the discourse are:
citation
Marvel
movies
year


In [153]:
print("Topic of every sentence:")
for utterance, utterance_topic in zip(utterances, topics_utterances):
    print(utterance, ":", utterance_topic)

Topic of every sentence:
By Marvel Entertainment began planning to independently produce its own films and distribute them through Paramount Pictures : 
Previously Marvel had co-produced several superhero films with Columbia Pictures New LieCiea and others including a seven-year development deal with th Century Fox : Marvel
Marvel made relatiey little profit from its liesing deals with other studios and wanted to get more money out of its films whiemaintaining artistic control of the projects and distribution : Marvel
Avi Arad head of Marvel s film division was pleased with Sam Raimi s Spie-Man films at Sony but was less pleased about others : Avi
As a result they decie to form Marvel Studios Hollywood s first major independent movie studio since DreamWorks : result
Arad s second-in-command Kevin Feie realie that unlieSpie-Man and the X-Men whose film rights were liesed to Sony and Fox respectiey Marvel still owned the rights to the core members of the Avengers : Arad
Feie a self-profe

In [154]:
for utterance, utterance_topic in zip(utterances, topics_utterances):
    processed_utterance = nlp(utterance)
    init_topic = ''
    for chunk in processed_utterance.noun_chunks:
        if chunk.root.dep_ == "nsubj":
            init_topic = chunk.text